In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# !pip install torchsummary
from torchsummary import summary

In [0]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 16, 3,  bias=False, padding=1)
#         self.conv2 = nn.Conv2d(16, 20, 3,  bias=False, padding=1)
#         self.conv3 = nn.Conv2d(20, 20, 3,  bias=False, padding=1)
#         self.batch_norm_16 = nn.BatchNorm2d(16)
#         self.batch_norm_20 = nn.BatchNorm2d(20)
#         self.batch_norm_10 = nn.BatchNorm2d(10)

#         self.dropout = nn.Dropout(0.10)


#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.reduce_channel = nn.Conv2d(20, 16, 1,  bias=False)

#         self.conv4 = nn.Conv2d(20, 10, 3,  bias=False)
#         self.conv5 = nn.Conv2d(16, 20, 3,  bias=False)
#         self.conv6 = nn.Conv2d(20, 20, 3,  bias=False)

#         self.AvgPool2d = nn.AvgPool2d(3)

#     def forward(self, x):
#         x = self.dropout(self.batch_norm_16(F.relu(self.conv1(x)))) #28
#         x = self.dropout(self.batch_norm_20(F.relu(self.conv2(x)))) #28
#         # x = self.dropout(self.batch_norm_20(F.relu(self.conv3(x)))) #28
#         x = self.reduce_channel(self.pool1(x)) #14
       
#         x = self.dropout(self.batch_norm_20(F.relu(self.conv2(x)))) #14
#         x = self.dropout(self.batch_norm_20(F.relu(self.conv3(x)))) #14
#         x = self.dropout(self.batch_norm_20(F.relu(self.conv3(x)))) #14
#         x = self.reduce_channel(self.pool1(x)) #7

#         x = self.dropout(self.batch_norm_20(F.relu(self.conv5(x)))) #5
#         x = self.dropout(self.batch_norm_20(F.relu(self.conv6(x)))) #3
#         # x = self.dropout(self.batch_norm_20(F.relu(self.conv6(x))))

#         x= self.conv4(x)
#         # x= self.AvgPool2d(x)



#         # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         # x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         # x = (self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1,8,3, padding =1,bias=False), #28
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(.10),

            nn.Conv2d(8,8,3, padding =1,bias=False), # 28
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(.10),

            nn.MaxPool2d(2,2), # 14
            # nn.Conv2d(20,16,1)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(8,16,3,bias=False), # 12
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(.10),

            nn.Conv2d(16,16,3,bias=False), # 10
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(.10),

            nn.MaxPool2d(2,2), #5
            # nn.Conv2d(20,16,1)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 24,3,bias=False), #3
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Dropout2d(.10),

            # nn.Conv2d(24,24,3,bias=False), #1
            # nn.ReLU(),
            # nn.BatchNorm2d(24),
            # nn.Dropout2d(.10),

            # nn.MaxPool2d(2,2), #5
            # nn.Conv2d(24,16,1)
        )

        self.reducer = nn.Conv2d(24,10,1,bias=False)
        self.gap_layer = nn.AvgPool2d(3)

    def forward(self, x):
      x= self.conv1(x)
      x= self.conv2(x)
      x= self.conv3(x)
      x= self.reducer(x)
      x= self.gap_layer(x)
      x = x.view(-1, 10)
      return F.log_softmax(x)

In [34]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
         Dropout2d-4            [-1, 8, 28, 28]               0
            Conv2d-5            [-1, 8, 28, 28]             576
              ReLU-6            [-1, 8, 28, 28]               0
       BatchNorm2d-7            [-1, 8, 28, 28]              16
         Dropout2d-8            [-1, 8, 28, 28]               0
         MaxPool2d-9            [-1, 8, 14, 14]               0
           Conv2d-10           [-1, 16, 12, 12]           1,152
             ReLU-11           [-1, 16, 12, 12]               0
      BatchNorm2d-12           [-1, 16, 12, 12]              32
        Dropout2d-13           [-1, 16, 12, 12]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [37]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
  print(f"epoch: {epoch} ")
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

epoch: 1 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0627, Accuracy: 9833/10000 (98.33%)

epoch: 2 


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0195, Accuracy: 9934/10000 (99.34%)

epoch: 19 


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0190, Accuracy: 9939/10000 (99.39%)

epoch: 20 


loss=0.008348315954208374 batch_id=937: 100%|██████████| 938/938 [00:17<00:00, 52.54it/s]



Test set: Average loss: 0.0192, Accuracy: 9946/10000 (99.46%)

